# ENSF 519 Final Project - Image to Caption Matcher
### Project Description: Our project outlined in the steps below tests 2 deep learning deep learning model's ability to match images to image captions using the Flickr8k Dataset, then intakes a defined image and displays the closest predicted captions.
Description: This project implements and compares two approaches for image-to-caption association: a pre-trained CLIP model (zero-shot) and a custom deep learning model trained on a Flickr-style dataset. The workflow includes loading and preprocessing images and captions, building a vocabulary, creating datasets and data loaders, defining image and text encoders (ResNet for images, BiLSTM for text), and training the custom model using a contrastive loss to align embeddings. After training, both models are evaluated on a held-out test set using retrieval metrics (Recall@1, Recall@5, Recall@10), and the results are compared. Finally, the project demonstrates the retrieval capability with a live demo image, retrieving the most relevant captions according to the models.
## Team Members:
- Gashagaza, Gisa Tchaka
- Ghattas, Mohamad
- Long, Alvin
- van Roessel, Spencer

How to run: To run the project, download the Flickr8k database, unzip the dataset, and place the "captions.txt" file as well as the "Images" folder into the same dirctory as the project file. The following packages need to be installed before running the code cells: 
- numpy 
- pillow 
- tqdm 
- torch 
- torchvision 
- transformers 
- nltk

## Step 0: Imports

In [ ]:
# Run in the folder that contains: captions.txt  and Images/ (Flickr8k)
# Requires: numpy, torch, torchvision, transformers, tqdm, nltk, pillow

import os, math, random
from pathlib import Path
from collections import defaultdict
import numpy as np
from PIL import Image
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import torchvision.transforms as T
import torchvision.models as models

# HuggingFace CLIP
from transformers import CLIPProcessor, CLIPModel

import nltk
nltk.download('punkt')

## Step 1: Setup Config / Paths / Device variables
Set up the core dataset paths and computing environment, ensure the model can access the images, captions, and uploaded demo image while selecting the appropriate device (CPU or GPU)

In [ ]:
DATA_DIR = Path('.')    # project folder with "captions.txt" and "Images" folder, currently in the project directory
CAPTIONS_FILE = DATA_DIR / 'captions.txt'
IMAGES_DIR = DATA_DIR / 'Images'
UPLOADED_IMAGE = "people-playing-foosball.png"  # uploaded image (for demo)
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", DEVICE)

# Step 2: Helpers: load captions (CSV style)
Load and structure the dataset captions by reading the captions file and map each image to its list of associated captions. Also prepare the text data in a convenient format for later training and evaluation of the image-to-caption models.

In [ ]:
def load_captions_csv(captions_file):
    # returns dict: image_filename -> list of captions
    img2caps = defaultdict(list)
    with open(captions_file, 'r', encoding='utf-8') as f:
        header = f.readline()  # skip header
        for line in f:
            line = line.strip()
            if not line:
                continue
            # split only on first comma
            img, cap = line.split(',', 1)
            img2caps[img].append(cap.strip())
    return dict(img2caps)

img2caps = load_captions_csv(CAPTIONS_FILE)
print("Images with captions:", len(img2caps))

## Step 3: Build splits (train/val/test)
Randomly split the dataset into training, validation, and tests sets to support proper model development and evaluation. Ensure that each model is trained on one subset of images while being fairly assessed on unseen data.

In [ ]:
all_images = sorted(list(img2caps.keys()))
random.seed(42)
random.shuffle(all_images)
N = len(all_images)
n_test = max(500, int(0.1 * N))   # test ~10% or 500 images (can adjust)
n_val = max(500, int(0.1 * N))
test_imgs = all_images[:n_test]
val_imgs = all_images[n_test:n_test+n_val]
train_imgs = all_images[n_test+n_val:]
print("Train/Val/Test sizes:", len(train_imgs), len(val_imgs), len(test_imgs))

## Step 4: Preprocessing transforms
Define the preprocessing pipeline for images, resizing them to a consistent size and normalizing their pixel values. Ensure that all images are formatted correctly and uniformly before being fed into the deep learning models.

In [ ]:
IMG_SIZE = 224
img_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225])
])

## Step 5: Dataset to produce image + single caption sample
Create a dataset that for each image returns a random caption among the 5. This stage defines a custom dataset class that loads image–caption pairs and applies preprocessing, enabling the model to receive training samples in a consistent format. It then creates the training, validation, and test datasets that will be used throughout the model development process.

In [ ]:
class FlickrPairsDataset(Dataset):
    def __init__(self, img_list, img2caps, images_dir, transform, max_text_len=30):
        self.img_list = img_list
        self.img2caps = img2caps
        self.images_dir = Path(images_dir)
        self.transform = transform
        self.max_text_len = max_text_len
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img_name = self.img_list[idx]
        img_path = self.images_dir / img_name
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        # choose one caption randomly for training sample
        caption = random.choice(self.img2caps[img_name])
        return image, caption, img_name

# quick datasets
train_ds = FlickrPairsDataset(train_imgs, img2caps, IMAGES_DIR, img_transform)
val_ds   = FlickrPairsDataset(val_imgs, img2caps, IMAGES_DIR, img_transform)
test_pairs = [(IMAGES_DIR / img, img2caps[img], img) for img in test_imgs]  # use all captions for retrieval

## Step 6: Utility: tokenize captions (simple nltk tokenizer)
A simple text-processing function that lowercases each caption and splits it into individual tokens using NLTK.

In [ ]:
def simple_tokenize(s):
    return nltk.word_tokenize(s.lower())

## 7a. Model A: CLIP baseline (HuggingFace)
- We will use CLIP to compute embeddings for all images and all captions
- Retrieval: cosine similarity between image and text embeddings

This stage loads the pre-trained CLIP model and its processor, setting it up on the appropriate device for inference. It also defines a function to compute normalized embeddings for images and captions in batches, which will be used for tasks like similarity computation or retrieval.

In [ ]:
print("Loading CLIP model...")
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(DEVICE)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
clip_model.eval()

@torch.no_grad()
def compute_clip_embeddings(image_paths, texts, batch_size=64):
    # image_paths: list of PIL images or tensors/paths
    # texts: list of text strings (same length) -- for dataset-wide embeddings
    # returns two arrays: image_embs (N, D), text_embs (N, D)
    # process images and texts separately
    # if image_paths are PIL paths, open them
    imgs = []
    for p in image_paths:
        if isinstance(p, (str, Path)):
            im = Image.open(p).convert('RGB').resize((IMG_SIZE, IMG_SIZE))
            imgs.append(im)
        else:
            imgs.append(p)  # already PIL or tensor
    # batch process images
    all_img_embs = []
    for i in range(0, len(imgs), batch_size):
        batch = imgs[i:i+batch_size]
        proc = clip_processor(images=batch, return_tensors='pt').to(DEVICE)
        out = clip_model.get_image_features(**proc)
        emb = out.detach()
        emb = F.normalize(emb, dim=-1)
        all_img_embs.append(emb.cpu())
    all_img_embs = torch.cat(all_img_embs, dim=0)
    # texts
    all_txt_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        proc = clip_processor(text=batch, return_tensors='pt', padding=True).to(DEVICE)
        out = clip_model.get_text_features(**proc)
        emb = F.normalize(out.detach(), dim=-1)
        all_txt_embs.append(emb.cpu())
    all_txt_embs = torch.cat(all_txt_embs, dim=0)
    return all_img_embs.numpy(), all_txt_embs.numpy()

## 7b. Model B: Custom model (ResNet18 + BiLSTM) to produce embeddings
- We'll use pretrained ResNet18 (optionally freeze), project to embed_dim
- Text encoder: word embedding + BiLSTM -> take last hidden (or mean) -> projection
- Train with symmetric contrastive (cross-entropy on similarity logits)

This stage defines custom deep learning encoders: a ResNet-based image encoder to extract visual features and a BiLSTM-based text encoder to convert tokenized captions into embeddings. Both models project their outputs into a shared embedding space and normalize them, enabling direct comparison between images and text.

In [ ]:
class ImageEncoderResNet(nn.Module):
    def __init__(self, embed_dim=512, train_resnet=True):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        # remove final fc
        self.backbone = nn.Sequential(*list(resnet.children())[:-1])    # outputs (B,512,1,1)
        if not train_resnet:
            for p in self.backbone.parameters():
                p.requires_grad = False
        self.proj = nn.Linear(resnet.fc.in_features, embed_dim)
    
    def forward(self, x):
        # x: (B,3,H,W)
        feat = self.backbone(x).squeeze(-1).squeeze(-1) # (B,512)
        out = self.proj(feat)                               # (B,embed_dim)
        out = F.normalize(out, dim=-1)
        return out

class TextEncoderBiLSTM(nn.Module):
    def __init__(self, vocab, embed_dim=512, hidden=512, num_layers=1):
        super().__init__()
        self.vocab = vocab
        self.embedding = nn.Embedding(len(vocab)+1, embedding_dim=300, padding_idx=0)
        self.lstm = nn.LSTM(input_size=300, hidden_size=hidden//2, num_layers=num_layers, batch_first=True, bidirectional=True)
        self.proj = nn.Linear(hidden, embed_dim)
    
    def forward(self, token_ids, lengths):
        # token_ids: (B, T)
        embedded = self.embedding(token_ids)            # (B,T,300)
        packed = nn.utils.rnn.pack_padded_sequence(embedded, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_out, (h_n, c_n) = self.lstm(packed)
        # h_n: (num_layers*2, B, hidden//2)
        h_cat = torch.cat([h_n[-2], h_n[-1]], dim=-1)   # (B, hidden)
        out = self.proj(h_cat)                          # (B, embed_dim)
        out = F.normalize(out, dim=-1)
        return out

## Step 8: Simple text tokenizer + vocab builder (small)
Build a vocabulary from the training captions, keeping words that meet a minimum frequency and assigning unique token IDs, with special handling for padding and unknown words. Also defines a function to convert captions into fixed-length sequences of token IDs for input into the text encoder.

In [ ]:
from collections import Counter
def build_vocab(img2caps, min_freq=2, max_size=10000):
    counter = Counter()
    for caps in img2caps.values():
        for c in caps:
            toks = simple_tokenize(c)
            counter.update(toks)
    words = [w for w,c in counter.items() if c >= min_freq]
    words = sorted(words, key=lambda w: -counter[w])[:max_size]
    stoi = {w:i+1 for i,w in enumerate(words)}  # 0 reserved for pad
    stoi['<unk>'] = len(stoi)+1
    itos = {i:w for w,i in stoi.items()}
    return {'stoi': stoi, 'itos': itos, 'pad': 0}

vocab = build_vocab({k:img2caps[k] for k in train_imgs}, min_freq=2)
print("Vocab size:", len(vocab['stoi']))

# convert a caption to token ids
def caption_to_ids(caption, stoi, max_len=30):
    toks = simple_tokenize(caption)[:max_len]
    ids = [stoi.get(t, stoi['<unk>']) for t in toks]
    length = len(ids)
    if length < max_len:
        ids = ids + [0] * (max_len - length)
    return ids, length

## Step 9: DataLoader for training custom model
Will yield batches of images and token_ids + lengths. Define a dataset loader for training that returns preprocessed images along with their tokenized caption IDs and lengths. Also provide a collate function to batch the samples efficiently for input into the model during training.

In [ ]:
class TrainPairsLoader(Dataset):
    def __init__(self, img_list, img2caps, images_dir, transform, stoi, max_len=30):
        self.img_list = img_list
        self.img2caps = img2caps
        self.images_dir = Path(images_dir)
        self.transform = transform
        self.stoi = stoi
        self.max_len = max_len
    
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img = self.img_list[idx]
        image = Image.open(self.images_dir / img).convert('RGB')
        image = self.transform(image)
        caption = random.choice(self.img2caps[img])
        ids, length = caption_to_ids(caption, self.stoi, max_len=self.max_len)
        return image, torch.tensor(ids, dtype=torch.long), length

def collate_train(batch):
    images, ids_list, lengths = zip(*batch)
    images = torch.stack(images, dim=0)
    ids = torch.stack(ids_list, dim=0)
    lengths = torch.tensor(lengths, dtype=torch.long)
    return images, ids, lengths

## Step 10: Contrastive loss training: symmetric cross-entropy on similarity logits
Define a contrastive loss function that measures how well image and text embeddings align in a shared space. En matching image–caption pairs to have high similarity while pushing non-matching pairs apart, using a temperature-scaled cross-entropy formulation.

In [ ]:
def contrastive_loss_from_embeddings(img_emb, txt_emb, temperature=0.07):
    # img_emb: (B,D), txt_emb: (B,D)
    logits = img_emb @ txt_emb.t()       # (B,B)
    logits = logits / temperature
    targets = torch.arange(logits.size(0), device=logits.device)
    loss_i = F.cross_entropy(logits, targets)
    loss_t = F.cross_entropy(logits.t(), targets)
    return (loss_i + loss_t) / 2.0

## Step 11: Training loop for custom model (small epochs for test)
Implement the training loop for the custom image and text encoders, computing embeddings, applying the contrastive loss, and updating model parameters via backpropagation. Iterate over the training dataset for multiple epochs, tracking and reporting the average loss to monitor learning progress.

In [ ]:
def train_custom_model(image_encoder, text_encoder, train_loader, optimizer, epochs=3):
    image_encoder.train(); text_encoder.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, ids, lengths in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            images = images.to(DEVICE)
            ids = ids.to(DEVICE)
            lengths = lengths.to(DEVICE)
            img_emb = image_encoder(images)                       # (B,D)
            txt_emb = text_encoder(ids, lengths)                  # (B,D)
            loss = contrastive_loss_from_embeddings(img_emb, txt_emb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f"Epoch {epoch+1} avg loss: {total_loss / len(train_loader):.4f}")

## Step 12: Retrieval evaluation metrics
Compute retrieval evaluation metrics to assess how well the image and text embeddings align. Calculate ranks of the correct captions for each image and reports Recall@1, Recall@5, Recall@10, and the median rank to quantify retrieval performance.
- Recall@1: The percentage of images where a correct caption was in the top 1 of the model predictions.
- Recall@5: The percentage of images where a correct caption was in the top 5 of the model predictions.
- Recall@10: The percentage of images where a correct caption was in the top 10 of the model predictions.

In [ ]:
def retrieval_metrics(image_embs, text_embs, img_names, text_to_img_map):
    # image_embs: (N_img, D), text_embs: (N_text, D)
    # img_names: list of image file names corresponding to image_embs
    # text_to_img_map: list mapping each text index -> image index
    sims = image_embs @ text_embs.T   # (N_img, N_text)
    # For each image, rank texts by similarity
    ranks = []
    for i in range(sims.shape[0]):
        sim_row = sims[i]
        sorted_idx = np.argsort(-sim_row)  # descending
        # find rank of the first *correct* text (any of the 5 captions)
        correct_text_inds = [j for j,t in enumerate(text_to_img_map) if t == i]
        # find best rank among correct_text_inds
        rank_positions = [np.where(sorted_idx == ct)[0][0] for ct in correct_text_inds]
        ranks.append(min(rank_positions) + 1)  # ranks are 1-based
    ranks = np.array(ranks)
    r1 = np.mean(ranks <= 1) * 100
    r5 = np.mean(ranks <= 5) * 100
    r10 = np.mean(ranks <= 10) * 100
    med = np.median(ranks)
    return {'R@1': r1, 'R@5': r5, 'R@10': r10, 'Median Rank': med}

## Step 13: Precompute caption list for retrieval
We'll prepare one caption per image entry in text_embs list (i.e., 5 captions per image). Prepare the test set for retrieval evaluation by collecting all captions and mapping each one to its corresponding image. Create lists of captions, their associated image indices, and image names to be used for computing similarity metrics.

In [ ]:
all_texts = []
text_to_img = []
text_img_names = []
for i,img in enumerate(test_imgs):
    caps = img2caps[img]
    for c in caps:
        all_texts.append(c)
        text_to_img.append(i)   # index in test_imgs
        text_img_names.append(img)

## Step 14: Evaluate CLIP baseline on test set
Evaluate the pre-trained CLIP model on the test set by computing embeddings for all test images and captions. Then, calculate retrieval metrics to measure how well CLIP matches images to their corresponding captions, providing a baseline performance.

In [ ]:
def eval_clip_on_test(test_imgs, all_texts, text_to_img):
    # prepare image list (PIL)
    img_paths = [IMAGES_DIR / img for img in test_imgs]
    # compute CLIP embs
    print("Computing CLIP embeddings...")
    img_embs, txt_embs = compute_clip_embeddings(img_paths, all_texts, batch_size=64)
    print("Image embs:", img_embs.shape, "Text embs:", txt_embs.shape)
    # compute retrieval metrics (image->text)
    metrics = retrieval_metrics(img_embs, txt_embs, test_imgs, text_to_img)
    return metrics

# Run CLIP baseline (this is faster because model is pretrained)
clip_metrics = eval_clip_on_test(test_imgs, all_texts, text_to_img)
print("CLIP baseline metrics (image->text):", clip_metrics)

## Step 15: Prepare and train custom model
Set up a smaller subset of the training data for fast experimentation, initialize the custom image and text encoders, and define the optimizer. Next, run a quick training loop to fine-tune the models on this subset, allowing for testing of the training process.

In [ ]:
# small subset for quick testing (comment out for full training)
subset_train_imgs = train_imgs[:2000]  # reduce if you want a fast run
train_loader = DataLoader(TrainPairsLoader(subset_train_imgs, img2caps, IMAGES_DIR, img_transform, vocab['stoi']),
                          batch_size=64, shuffle=True, collate_fn=collate_train)

embed_dim = 512
image_encoder = ImageEncoderResNet(embed_dim=embed_dim, train_resnet=True).to(DEVICE)
text_encoder = TextEncoderBiLSTM(vocab=vocab['stoi'], embed_dim=embed_dim, hidden=512).to(DEVICE)

optimizer = torch.optim.Adam(list(image_encoder.parameters()) + list(text_encoder.parameters()), lr=1e-4)

# Quick train (few epochs)
train_custom_model(image_encoder, text_encoder, train_loader, optimizer, epochs=10)  # Currently only running for 2 epochs

## Step 16: Compute embeddings for test set for custom model
Compute embeddings for the test images and captions using the fine-tuned custom image and text encoders. Evaluate the model’s retrieval performance on the test set by calculating metrics, providing a comparison to the CLIP baseline.

In [ ]:
@torch.no_grad()
def compute_custom_embeddings(image_paths, texts, image_encoder, text_encoder, stoi, batch_size=64, max_len=30):
    # image_paths: list of image Path objects
    image_embs = []
    for i in range(0, len(image_paths), batch_size):
        batch_paths = image_paths[i:i+batch_size]
        batch_imgs = [Image.open(p).convert('RGB') for p in batch_paths]
        batch_imgs = [img_transform(im) for im in batch_imgs]
        batch = torch.stack(batch_imgs).to(DEVICE)
        emb = image_encoder(batch).cpu()
        image_embs.append(emb)
    image_embs = torch.cat(image_embs, dim=0).numpy()
    
    text_embs = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        ids = []
        lengths = []
        for t in batch:
            idseq, L = caption_to_ids(t, stoi, max_len=max_len)
            ids.append(idseq)
            lengths.append(L)
        ids = torch.tensor(ids, dtype=torch.long).to(DEVICE)
        lengths = torch.tensor(lengths).to(DEVICE)
        emb = text_encoder(ids, lengths).cpu()
        text_embs.append(emb)
    text_embs = torch.cat(text_embs, dim=0).numpy()
    return image_embs, text_embs

print("Computing custom model embeddings (test set)...")
img_paths = [IMAGES_DIR / img for img in test_imgs]
custom_img_embs, custom_txt_embs = compute_custom_embeddings(img_paths, all_texts, image_encoder, text_encoder, vocab['stoi'])
custom_metrics = retrieval_metrics(custom_img_embs, custom_txt_embs, test_imgs, text_to_img)
print("Custom model metrics (image->text):", custom_metrics)

## Step 17: Compare and print results
This stage summarizes and compares the retrieval performance of the pre-trained CLIP model and the fine-tuned custom model. It also prints the key metrics (Recall@1, Recall@5, Recall@10) to highlight how training improves image-to-caption matching.

In [ ]:
print("=== Results ===")
print("CLIP (zero-shot) R@1/5/10:", clip_metrics['R@1'], clip_metrics['R@5'], clip_metrics['R@10'])
print("Custom (trained) R@1/5/10:", custom_metrics['R@1'], custom_metrics['R@5'], custom_metrics['R@10'])

# Step 18: Quick qualitative demo with uploaded image
As an add-on, this step demonstrates the CLIP model’s zero-shot retrieval by displaying an uploaded image and finding the top matching captions from a subset of test captions. It computes the image embedding, compares it to precomputed text embeddings, and outputs the highest-similarity captions for visualization.

In [ ]:
from IPython.display import display
display(Image.open(UPLOADED_IMAGE))  # show uploaded image

# Use CLIP to retrieve top captions among all_texts for that uploaded image
proc = clip_processor(images=Image.open(UPLOADED_IMAGE).convert('RGB').resize((IMG_SIZE, IMG_SIZE)), return_tensors='pt').to(DEVICE)
with torch.no_grad():
    im_feat = clip_model.get_image_features(**proc)
    im_feat = F.normalize(im_feat, dim=-1).cpu().numpy()  # (1,D)
# compare to CLIP text embeddings we computed earlier (txt_embs from eval_clip_on_test)
# Note: compute_clip_embeddings returns arrays; reload if needed. For demo, recompute small set:
demo_texts = all_texts[:500]   # demo subset
_, demo_txt_embs = compute_clip_embeddings([IMAGES_DIR / i for i in test_imgs[:500]], demo_texts, batch_size=64)
sims = (im_feat @ demo_txt_embs.T).squeeze()
topk = sims.argsort()[::-1][:10]
print("Top CLIP captions:")
for k in topk:
    print(demo_texts[k])